In [ ]:
import pandas as pd
import numpy as np

In [ ]:
X = pd.read_excel('../YouScienceData/Skyward_Blackman Middle.xlsx')
print(len(X))
X.head()

In [ ]:
Y = pd.read_csv('../YouScienceData/YouScience_cluster_advising_Blackman Middle School.csv')
print(len(set(Y.id)))
Y.head()

In [ ]:
# Checking YouScience csv for those not in skyward
Z = Y.drop_duplicates('email')
not_in_skyward = []
i = 0
for email in Z.email:
    check = X.loc[X['Student\'s School Email'] == email]
    # check if missing student
    if len(check) == 0:
        student = Z.loc[Z.email == email]
        first, last = student['first_name'], student['last_name']
        not_in_skyward.append([first, last])
        i += 1
print(f'YouScience Roster not in skyward: {i}') 
print(f'Length of Skyward Roster: {len(X)}')

In [ ]:
# checking skyward for those not in Youscience
not_in_YS = []
j = 0
for email in X['Student\'s School Email']:
    check = Z.loc[Z.email == email]
    if len(check) == 0:
        student = X.loc[X['Student\'s School Email'] == email]
        first, last = student['Student First Name'], student['Student Last Name']
        not_in_YS.append([first, last])
        j += 1
print(f'Skyward Roster not in YouScience: {j}')
print(f'Length of Skyward Roster: {len(X)}')
print(f'Length of overlapping roster: {len(X)-j}')

In [ ]:
# checking if there are any missing emails in either df
sky_missing = len(X.loc[X['Student\'s School Email'] == ''])
ys_missing = len(Z.loc[Z.email == ''])
print(sky_missing, ys_missing)

In [ ]:
def get_breakdown_by_rank(df, rank=1):
    # compiling frequency of interest/aptitude pairs
    
    fits = list(set(df.aptitude_fit).union(df.interest_fit))
    df = df[df.fit_rank == rank]
    
    breakdown_df = None
    for fit in fits:
        Y = df.loc[X.aptitude_fit == fit].interest_fit.value_counts()
        Y = pd.DataFrame(Y)
        Y.rename(columns={'interest_fit':'Apt: ' + fit},inplace=True)

        if breakdown_df is None: 
            breakdown_df = Y
        else:
            breakdown_df = pd.concat([breakdown_df,Y], axis=1)

    breakdown_df.replace(np.nan,0,inplace=True)
    
    return breakdown_df

In [ ]:
r1_df = get_breakdown_by_rank(X)
util_dict = {}
for col in list(r1_df.columns):
    total = int(r1_df[col].sum())
    try:
        cat = r1_df[col].loc[col.split(': ')[1]]
        print(col, '- Total: ', total, '; %Interested: ', "{:.2%}".format(float(cat)/total))
    except KeyError:
        print(f'~~~At fit_rank 1, none were labeled with {col}')

In [ ]:
list(r1_df.columns)


In [ ]:
# checking no students were dropped 
sum = 0
for col in list(r1_df.columns):
    sum += r1_df[col].sum()
sum

In [ ]:
# how many students across RCS had interest/aptitude align with fit_rank 1? 
perfect_fit = 0
for col in list(r1_df.columns):
    try:
        perfect_fit += int(r1_df[col].loc[col.split(': ')[1]])   
    except KeyError:
        pass

print(perfect_fit, 9309-perfect_fit, "{:.2%}".format(perfect_fit/7960))

# This does not mean that those 1349 could get into their appropriate POS placements

In [ ]:
list(set(X.organization))

In [ ]:
Z = X[X.organization == 'LaVergne Middle School']
# compiling frequency of interest/aptitude pairs
fits = list(set(X.aptitude_fit).union(X.interest_fit))
breakdown_df = None
for fit in fits:
    Y = Z.loc[Z.aptitude_fit == fit].interest_fit.value_counts()
    Y = pd.DataFrame(Y)
    Y.rename(columns={'interest_fit':'Apt: ' + fit},inplace=True)

    if breakdown_df is None: 
        breakdown_df = Y
    else:
        breakdown_df = pd.concat([breakdown_df,Y], axis=1)

breakdown_df.replace(np.nan,0,inplace=True)

breakdown_df

In [ ]:
int_total, apt_total = {}, {}
print('Interest | Aptitude | Category')
for fit in fits:
    int_total[fit] = breakdown_df.loc[fit].sum()
    apt_total['Apt: ' + fit] = breakdown_df['Apt: ' + fit].sum()
    
    print(int_total[fit], apt_total['Apt: ' + fit], fit)